In [1]:
import os
import random
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

# EDA

In [2]:
import json
import pandas as pd

In [3]:
path = r"C:\Users\esual\Downloads\seminar_fma_cl\data\eli5"
file = 'generate_a_question_1.train_asks.json'

# Load data
with open(os.path.join(path, file), 'r') as f:
    data = json.load(f)

In [4]:
# Convert to DataFrame
df = pd.DataFrame(data)
df.head()

,src,tgt,src_info
0,I did some research as a layman. One forum use...,What are the long-term effects of using raid?,{'question': 'What are the long-term effects o...
1,Marine biologists and geologists often have op...,"Hey reddit, what are some jobs in science that...","{'question': 'Hey reddit, what are some jobs i..."
2,"It's related to the ""beat"" period/frequency - ...","What is the proper name for a period T, when t...",{'question': 'What is the proper name for a pe...
3,Write an inquisitive question about the follow...,When two different versions of the same word c...,{'question': 'When two different versions of t...
4,"Given the following text, what could have been...","How big do people from ""pygmy"" groups get when...","{'question': 'How big do people from ""pygmy"" g..."


In [12]:
df.shape

(61710, 3)

In [14]:
df.iloc[0,'src_info']

src         I did some research as a layman. One forum use...
tgt             What are the long-term effects of using raid?
src_info    {'question': 'What are the long-term effects o...
Name: 0, dtype: object

In [15]:
df.query('tgt == "What are the long-term effects of using raid?"')

,src,tgt,src_info
0,I did some research as a layman. One forum use...,What are the long-term effects of using raid?,{'question': 'What are the long-term effects o...


# Create dataset

In [5]:
path_data = r"C:\Users\esual\Downloads\seminar_fma_cl\data"

In [ ]:
import json
import random
random.seed(666)

mapper = lambda x: 'validation' if x in {'quail', 'social_i_qa', 'quoref', 'validation'} else 'test'

#d_t0_train_datasets = {name: {mapper(name): [('t0_template', "__RANDOM__")]} for name in t0_train_datasets if name not in skip}
#process_datasets(d_t0_train_datasets, limit_nb_examples=1000, path_data=path_data)

all_data = {'src':[], 'tgt':[], 'src_info':[]}
for name_dataset in [
        'qasc', 'quarel', 'sciq', 'cosmos_qa', 'wiqa', 'social_i_qa', 'quartz',  'quail', 'adversarial_qa_dbert', 'adversarial_qa_dbidaf',
        'adversarial_qa_droberta', 'duorc_ParaphraseRC', 'duorc_SelfRC', 'quoref', 'wiki_qa', 'common_gen', 'imdb', 'xsum', 'ag_news',
        'rotten_tomatoes', 'glue_qqp', 'trec', 'paws_labeled_final', 'glue_mrpc', 'dream'
  ]:

  path = f"{path_data}/{name_dataset}/__RANDOM__.{mapper(name_dataset)}.json"

  with open(path, 'r') as f:
    data = json.load(f)

    

  for k in ['src', 'tgt', 'src_info']:
    all_data[k] += data[k]

sampled_ids = random.sample(list(range(len(all_data[k]))), k=1000)

sampled_data = {}
for k in ['src', 'tgt', 'src_info']:
  sampled_data[k] = [all_data[k][i] for i in sampled_ids]

path = f"{path_data}/T0_train_eval/__RANDOM__.test.json"
with open(path, 'w') as f:
  json.dump(sampled_data, f)

In [14]:
all_data['src_info'][10000]

{'id': 'bb7faa8846f5bf96f63e956d8b0dccaa36fd2236',
 'title': 'Yuan_dynasty',
 'context': 'Many Han Chinese and Khitan defected to the Mongols to fight against the Jin. Two Han Chinese leaders, Shi Tianze, Liu Heima (劉黑馬, Liu Ni), and the Khitan Xiao Zhala (蕭札剌) defected and commanded the 3 Tumens in the Mongol army. Liu Heima and Shi Tianze served Ogödei Khan. Liu Heima and Shi Tianxiang led armies against Western Xia for the Mongols. There were 4 Han Tumens and 3 Khitan Tumens, with each Tumen consisting of 10,000 troops. The three Khitan Generals Shimobeidier (石抹孛迭兒), Tabuyir (塔不已兒) and Xiaozhacizhizizhongxi (蕭札刺之子重喜) commanded the three Khitan Tumens and the four Han Generals Zhang Rou, Yan Shi, Shi Tianze, and Liu Heima commanded the four Han tumens under Ogödei Khan.',
 'question': 'Who was lower in rank, Ogodei Khan or Liu Heima?',
 'answers': {'text': [], 'answer_start': []},
 'metadata': {'split': 'test', 'model_in_the_loop': 'RoBERTa-Large'},
 'prompt_name': 'generate_question

In [16]:
len(all_data['src'])

23790

In [17]:
mapper = lambda x: 'validation' if x == 'asset' else ('train_asks' if x == 'eli5' else 'train')

all_data = {'src':[], 'tgt':[], 'src_info':[]}
for name_dataset in [
        'wiki_auto', # Simp
        'asset', # Simp
        'eli5', # InqQG
        'eSNLI', # Exp
        'empathetic_dialogues', # Emdg
        'gigaword' # HGen
  ]:

  path = f"{path_data}/{name_dataset}/__RANDOM__.{mapper(name_dataset)}.json"

  with open(path, 'r') as f:
    data = json.load(f)

    if data is None:
      print(f"Skipping {name_dataset}")
      continue

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\esual\\Downloads\\seminar_fma_cl\\data/wiki_auto/__RANDOM__.train.json'

In [18]:
from t0_continual_learning import model_prediction
from t0_continual_learning.config_variables import evaluation_new_tasks, evaluation_T0evalsets

#path_data = f'{GLOBAL_PATH}/data'
#path_pred = f'{GLOBAL_PATH}/predictions'

d_datasets = {
    **evaluation_new_tasks,
    **evaluation_T0evalsets
}

c:\Users\esual\miniconda3\envs\clearning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
d_datasets.keys()

dict_keys(['gigaword', 'wiki_auto', 'asset', 'eli5', 'haiku', 'covid_qa_deepset', 'empathetic_dialogues', 'eSNLI', 'twitter_top20', 'T0_train_eval', 'winogrande', 'rte', 'wic', 'copa', 'hellaswag', 'anli', 'cb', 'wsc', 'story_cloze'])

In [22]:
d_datasets['wiki_auto']

{'test': {'simplification_1': {'type': 'custom',
   'choice': '',
   'metrics': ['bleu', 'sari']}}}

# only wikiauto

In [1]:
import os
import numpy as np
import json
import torch
import pandas as pd

#import evaluate
from datasets import load_dataset
import random

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)

In [2]:
torch.cuda.is_available()

True

In [3]:
''.join(['../data/', 'wiki_auto'])

'../data/wiki_auto'

In [4]:
name_dataset = 'wiki_auto'
path = f"../data/raw/{name_dataset}/simplification_1.test.json"
with open(path, 'r') as f:
  data = json.load(f)

In [5]:
data

{'src': ['Reformulate this text with simpler words: "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D \'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .\n"',
  'Make this text simpler: "The new colonists named their settlement after the Canton of Bern , Switzerland , home state of their patron .\n"',
  'Make this text simpler: "New Bern is the second-oldest European settled colonial town in North Carolina , after Bath .\n"',
  'It served as the capital of the North Carolina colonial government , then briefly as the state capital .\n\n\n===\n\nGiven the above text, reformulate it so it is easier to read:',
  'Reformulate the following text with simpler words: "After the American Revolution , New Bern became wealthy and quickly developed a rich cultural life .\n"',
  'At one time New Bern was called " the Athens of the South , " renowned for its Masonic Temple and Athens Theater .\n\n\n===\n\nGiven

In [6]:
data['src']

['Reformulate this text with simpler words: "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D \'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .\n"',
 'Make this text simpler: "The new colonists named their settlement after the Canton of Bern , Switzerland , home state of their patron .\n"',
 'Make this text simpler: "New Bern is the second-oldest European settled colonial town in North Carolina , after Bath .\n"',
 'It served as the capital of the North Carolina colonial government , then briefly as the state capital .\n\n\n===\n\nGiven the above text, reformulate it so it is easier to read:',
 'Reformulate the following text with simpler words: "After the American Revolution , New Bern became wealthy and quickly developed a rich cultural life .\n"',
 'At one time New Bern was called " the Athens of the South , " renowned for its Masonic Temple and Athens Theater .\n\n\n===\n\nGiven the above te

In [7]:
# Find 'Robert Alda' in the data['src]
for i, src in enumerate(data['src']):
    if 'Alda ' in src:
        print(i, src)
        break
    else:
        print(i, '---')

0 Reformulate this text with simpler words: "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D 'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .
"


In [8]:
data.keys()

dict_keys(['src', 'tgt', 'src_info'])

In [9]:
data_df = pd.DataFrame(data)

In [12]:
rand_ind = random.randint(0, len(data_df))
print(data_df.iloc[rand_ind,2])
print('====================')
print(data_df.iloc[rand_ind,1])
print('====================')
print(data_df.iloc[rand_ind,0])

{'normal_sentence': 'In 2015 it caused around 3,500 deaths , down from 34,000 in 1990 .\n', 'simple_sentence': 'As of 2010 , sleeping sickness had caused around 9,000 deaths per year -LRB- compared to 34,000 in 1990 -RRB- .\n', 'prompt': '[normal_sentence]\n\n===\n\nGiven this text, write it with, simpler words:', 'prompt_name': 'simplification_1', 'prompt_mode': 'custom'}
As of 2010 , sleeping sickness had caused around 9,000 deaths per year -LRB- compared to 34,000 in 1990 -RRB- .

In 2015 it caused around 3,500 deaths , down from 34,000 in 1990 .


===

Given this text, write it with, simpler words:


In [15]:
data_df.iloc[0,2]

{'normal_sentence': "His father Robert Alda -LRB- born Alphonso Giuseppe Giovanni Roberto D 'Abruzzo -RRB- was an actor and singer , and his mother Joan Browne was a homemaker and former beauty-pageant winner .\n",
 'simple_sentence': 'His father , Robert Alda , was an actor , and his mother , Joan , was once crowned Miss New York .\n',
 'prompt': 'Reformulate this text with simpler words: "[normal_sentence]"',
 'prompt_name': 'simplification_1',
 'prompt_mode': 'custom'}

In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
os.environ['TOKENIZERS_PARALLELISM'] = 'false' # there might be interferences with the parallelism of the Hugging Face Trainer

In [9]:
model_name = "llm-stacking/StackLLM_3B_300BToken"  # Replace with the name of your pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Users\esual\miniconda3\envs\clearning\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
data_rand = data_df.iloc[rand_ind]

In [11]:
data_rand

src         The water depth is 57 m and at 100 % capacity ...
tgt                            Normal water depth is 57 m .\n
src_info    {'normal_sentence': 'The water depth is 57 m a...
Name: 39520, dtype: object

In [12]:
tokenizer(data_rand["src"], truncation=True, max_length=512)

{'input_ids': [1, 450, 4094, 10809, 338, 29871, 29945, 29955, 286, 322, 472, 29871, 29896, 29900, 29900, 1273, 13284, 278, 4094, 3233, 338, 29871, 29941, 29946, 29946, 286, 319, 26124, 869, 13, 13, 13, 25512, 13, 13, 29954, 5428, 445, 1426, 29892, 2436, 372, 411, 29892, 13682, 3838, 29901], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer(data_rand["tgt"], truncation=True, max_length=128)

{'input_ids': [1, 21981, 4094, 10809, 338, 29871, 29945, 29955, 286, 869, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
def preprocess_simpl(data):
    tokenized_data = tokenizer(data['src'], truncation=True, max_length=512)
    tokenized_labels = tokenizer(data['tgt'], truncation=True, max_length=128)
    tokenized_data['labels'] = tokenized_labels['input_ids']
    return tokenized_data

In [15]:
type(data['src'])

list

In [16]:
tokenized_wiki = data_df.apply(preprocess_simpl, axis=1)

In [17]:
tokenized_wiki.iloc[10]

{'input_ids': [1, 23856, 1306, 29871, 30478, 341, 6637, 3219, 493, 1794, 448, 29519, 29933, 29899, 11324, 29871, 30712, 29871, 30511, 29874, 29871, 30712, 29871, 30176, 31513, 30234, 448, 2227, 4162, 29899, 286, 448, 2227, 4162, 29899, 29871, 30426, 30215, 448, 2227, 4162, 29899, 29871, 30176, 448, 2227, 4162, 29899, 260, 448, 2227, 4162, 29899, 263, 30312, 2056, 29871, 29896, 3786, 29871, 29896, 29929, 29946, 29900, 785, 29871, 29906, 29945, 3839, 29871, 29906, 29900, 29896, 29896, 448, 29934, 29934, 29933, 29899, 471, 263, 4325, 26689, 10015, 10094, 5264, 1919, 29380, 322, 8604, 5039, 391, 322, 278, 937, 11715, 6114, 304, 5401, 278, 27813, 20604, 869, 13, 13, 13, 25512, 13, 13, 29954, 5428, 278, 2038, 1426, 29892, 2436, 372, 411, 29892, 13682, 3838, 29901], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [18]:
model = AutoModelForCausalLM.from_pretrained(model_name)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at llm-stacking/StackLLM_3B_300BToken and are newly initialized: ['model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn

In [19]:
# checking if model exists
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2560, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=512, bias=False)
          (v_proj): Linear(in_features=2560, out_features=512, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2560, out_features=7040, bias=False)
          (up_proj): Linear(in_features=2560, out_features=7040, bias=False)
          (down_proj): Linear(in_features=7040, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [2]:
test_data = load_dataset('tatsu-lab/alpaca', trust_remote_code=True)

In [3]:
test_data = test_data['train']
test_data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [11]:
# Choose random sample
sample = random.choice(test_data)

# Print one sample from dataset
print(f'''
Instruction: {sample['instruction']}
Input: {sample['input']}
Output: {sample['output']}
Text: {sample['text']}


''')

NameError: name 'test_data' is not defined

In [7]:
sample

{'instruction': 'Find the words in the sentence which suggest a positive attitude',
 'input': 'I think I can do this!',
 'output': '"I think", "can do"',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nFind the words in the sentence which suggest a positive attitude\n\n### Input:\nI think I can do this!\n\n### Response:\n"I think", "can do"'}

In [13]:
sampled_data

NameError: name 'sampled_data' is not defined

In [4]:
name_dataset = 'wiki_auto'
path = f"../data/raw/{name_dataset}/simplification_1.test.json"

# read json file with pd.read_json
df = pd.read_json(path)

In [44]:
# read all json files in ../data/raw/wiki_auto and merge them
path = f"../data/raw/{name_dataset}"
all_files = os.listdir(path)
li = []

for filename in all_files:
    if filename.endswith('.json'):
        file_path = os.path.join(path, filename)
        data = pd.read_json(file_path)
        data['source'] = filename
        li.append(data)

df = pd.concat(li, axis=0, ignore_index=True)

In [45]:
# find the row where src contains 'Alda ' 
df[df['src'].str.contains('Alda ')].to_clipboard()

In [46]:
df.head()

,src,tgt,src_info,source
0,"Reformulate this text with simpler words: ""His...","His father , Robert Alda , was an actor , and ...",{'normal_sentence': 'His father Robert Alda -L...,simplification_1.test.json
1,"Make this text simpler: ""The new colonists nam...",It was named after the capital of Switzerland ...,{'normal_sentence': 'The new colonists named t...,simplification_1.test.json
2,"Make this text simpler: ""New Bern is the secon...",New Bern is the second oldest town in North Ca...,{'normal_sentence': 'New Bern is the second-ol...,simplification_1.test.json
3,It served as the capital of the North Carolina...,It was the capital of the North Carolina colon...,{'normal_sentence': 'It served as the capital ...,simplification_1.test.json
4,Reformulate the following text with simpler wo...,"After the American Revolution , New Bern becam...",{'normal_sentence': 'After the American Revolu...,simplification_1.test.json


In [43]:
print('df size:', df.shape)
print('df unique size:', df['tgt'].nunique())

df size: (108002, 3)
df unique size: 102994


In [48]:
df.source.value_counts()

source
simplification_1.train.json    100002
simplification_1.test.json       4000
simplification_2.test.json       4000
Name: count, dtype: int64

In [50]:
df.rename(columns = {'src': 'input', 'tgt': 'output'}, inplace=True)
# save df as json to the same folder
df.to_json(path + '/simplification_man.json')

In [54]:
df.query('source != "simplification_1.train.json"').drop_duplicates(subset='output')

,input,output,src_info,source
0,"Reformulate this text with simpler words: ""His...","His father , Robert Alda , was an actor , and ...",{'normal_sentence': 'His father Robert Alda -L...,simplification_1.test.json
1,"Make this text simpler: ""The new colonists nam...",It was named after the capital of Switzerland ...,{'normal_sentence': 'The new colonists named t...,simplification_1.test.json
2,"Make this text simpler: ""New Bern is the secon...",New Bern is the second oldest town in North Ca...,{'normal_sentence': 'New Bern is the second-ol...,simplification_1.test.json
3,It served as the capital of the North Carolina...,It was the capital of the North Carolina colon...,{'normal_sentence': 'It served as the capital ...,simplification_1.test.json
4,Reformulate the following text with simpler wo...,"After the American Revolution , New Bern becam...",{'normal_sentence': 'After the American Revolu...,simplification_1.test.json
...,...,...,...,...
3995,"Yamagata -LRB- 山形市 , Yamagata-shi , -RSB- -RRB...","Yamagata -LRB- 山形市 , Yamagata-shi -RRB- is a c...","{'normal_sentence': 'Yamagata -LRB- 山形市 , Yama...",simplification_1.test.json
3996,"Yamagata -LRB- 山形市 , Yamagata-shi , -RSB- -RRB...",It is the capital city of Yamagata Prefecture .\n,"{'normal_sentence': 'Yamagata -LRB- 山形市 , Yama...",simplification_1.test.json
3997,The city attained special city status on April...,It has been recognized as a special city since...,{'normal_sentence': 'The city attained special...,simplification_1.test.json
3998,"Generate a text easier to understand, with sim...",Following the Protestant Reformation the theor...,{'normal_sentence': 'With the rise of nation-s...,simplification_1.test.json


In [16]:
name_dataset = 'wiki_auto'
path = f"../data/preprocessed/simplification_man.json"
with open(path, 'r') as f:
  data = json.load(f)

In [18]:
pd.DataFrame(data)

,input,output,src_info,source
0,"Reformulate this text with simpler words: ""His...","His father , Robert Alda , was an actor , and ...",{'normal_sentence': 'His father Robert Alda -L...,simplification_1.test.json
1,"Make this text simpler: ""The new colonists nam...",It was named after the capital of Switzerland ...,{'normal_sentence': 'The new colonists named t...,simplification_1.test.json
2,"Make this text simpler: ""New Bern is the secon...",New Bern is the second oldest town in North Ca...,{'normal_sentence': 'New Bern is the second-ol...,simplification_1.test.json
3,It served as the capital of the North Carolina...,It was the capital of the North Carolina colon...,{'normal_sentence': 'It served as the capital ...,simplification_1.test.json
4,Reformulate the following text with simpler wo...,"After the American Revolution , New Bern becam...",{'normal_sentence': 'After the American Revolu...,simplification_1.test.json
...,...,...,...,...
107997,"Yamagata -LRB- 山形市 , Yamagata-shi , -RSB- -RRB...","Yamagata -LRB- 山形市 , Yamagata-shi -RRB- is a c...","{'normal_sentence': 'Yamagata -LRB- 山形市 , Yama...",simplification_2.test.json
107998,"Yamagata -LRB- 山形市 , Yamagata-shi , -RSB- -RRB...",It is the capital city of Yamagata Prefecture .\n,"{'normal_sentence': 'Yamagata -LRB- 山形市 , Yama...",simplification_2.test.json
107999,"Simplify this text: ""The city attained special...",It has been recognized as a special city since...,{'normal_sentence': 'The city attained special...,simplification_2.test.json
108000,With the rise of nation-states and the Protest...,Following the Protestant Reformation the theor...,{'normal_sentence': 'With the rise of nation-s...,simplification_2.test.json


In [22]:
name_dataset = 'wiki_auto'
path = f"../data/preprocessed/wiki_auto_simp.json"
with open(path, 'r') as f:
  data = json.load(f)

In [24]:
data['train'][0]

{'input': 'Make this text simpler: "" Note : Some countries may occasionally claim to be " neutral " but not comply with the internationally agreed upon definition of neutrality as listed above . "\n"',
 'output': '" Note : Whether a state that is a member of the European Union may be considered neutral is a point of debate .\n',
 'src_info': {'normal_sentence': '" Note : Some countries may occasionally claim to be " neutral " but not comply with the internationally agreed upon definition of neutrality as listed above . "\n',
  'prompt': 'Make this text simpler: "[normal_sentence]"',
  'prompt_mode': 'custom',
  'prompt_name': 'simplification_1',
  'simple_sentence': '" Note : Whether a state that is a member of the European Union may be considered neutral is a point of debate .\n'},
 'index': 89061}

In [21]:
import pandas as pd
df=pd.read_json(path, lines=True)
df.head()


ValueError: Expected object or value

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained causal language model and tokenizer
model_name = "gpt2"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Input prompt
input_prompt = "Once upon a time, in a distant land, there was a"

# Tokenize the input prompt
inputs = tokenizer(input_prompt, return_tensors="pt")

# Generate predictions
outputs = model.generate(
    inputs["input_ids"],
    max_length=50,  # Maximum length of generated text
    num_beams=5,    # Beam search for better output quality
    no_repeat_ngram_size=2,  # Avoid repeating phrases
    temperature=0.7  # Control randomness of the output
)

# Decode the generated tokens
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated text
print("Generated text:", generated_text)


c:\Users\esual\miniconda3\envs\temp_dl4nlp\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\esual\miniconda3\envs\temp_dl4nlp\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any fe

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

c:\Users\esual\miniconda3\envs\temp_dl4nlp\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\esual\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

c:\Users\esual\miniconda3\envs\temp_dl4nlp\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text: Once upon a time, in a distant land, there was a man who had been sent by God to save the world. He was the son of God, and he was called the Son of Man. And when he came to the land of Israel


In [ ]:
outputs